In [19]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [6]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [118]:
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table')

In [132]:
data_rows = table.findAll('tr')[1:]

In [179]:
codes = []
for i in range(len(data_rows)):
    row = []
    for td in data_rows[i].findAll('td'):
        row.append(td.getText())
    codes.append(row)

In [148]:
col = ['PostalCode', 'Borough', 'Neighborhood']
postal_codes = pd.DataFrame(codes, columns=col)
postal_codes.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [216]:
postal_codes1 = postal_codes[postal_codes['Borough'] != 'Not assigned'].reset_index(drop=True)
postal_codes1.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods\n
1,M4A,North York,Victoria Village\n
2,M5A,Downtown Toronto,Harbourfront\n
3,M5A,Downtown Toronto,Regent Park\n
4,M6A,North York,Lawrence Heights\n


In [225]:
postal_codes1['Neighborhood'] = postal_codes1.loc[:,'Neighborhood'].str.strip()
postal_codes1.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
